In [512]:
import json
import requests
import pandas as pd
import numpy as np
import wget

#     <center> HOMEWORK 6  </center>

### En un notebook de jupyter desarrolle los siguientes pasosDescargue el JSON con la lista de paises del siguiente link:https://datahub.io/core/country-list/r/data.jsony escoja aleatoriamente un país de Europa

In [513]:
# Descargando el JSON
filename = wget.download('https://pkgstore.datahub.io/core/country-list/data_json/data/8c458f2d15d9f2119654b29ede6e45b8/data_json.json')

100% [............................................................] 9576 / 9576

In [514]:
#Abriendo el archivo descargado y cargandolo como json
paises_json = json.load(open('data_json.json'))

In [515]:
#Eligiendo aleatoriamente uno de los paises del json
tamano_json= len(paises_json)
index_elegido = np.random.randint(tamano_json) 
pais = paises_json[index_elegido]['Name'] 
print("El país elegido aleatoriamente es:", pais)

El país elegido aleatoriamente es: Réunion


In [516]:
#Para un pais en particular 
pais='Finland'
#pais='Italia'
#pais='Germany'

### Use los diferentes API endpoints de inspire-hep ' https://inspirehep.net/ '  Para extraer la lista de investigadores de una institución de ese país  en esa base de datos. Para ello:

### A) Use el API de institutions para extraer la lista de instituciones del país, por ejemplo: colombia (mejor si es Europa, por completo de datos ) https://inspirehep.net/api/institutions?q=colombia

In [517]:
institution_link = requests.get("https://inspirehep.net/api/institutions?q={}".format(pais))

In [518]:
## Extrayendo la lista de las instituciones del pais 
inst_names= [institution.get('metadata').get('legacy_ICN') for institution in institution_link.json().get('hits').get('hits')]
print("Las instituciones de ese país son:",inst_names)

Las instituciones de ese país son: ['Helsinki, Academy of Finland', 'UEF, Kuopio', 'Unlisted, FI', 'VALMET Automation, Tampere', 'Nevanlinna Inst., Helsinki', 'IVO Intl., Helsinki', 'VTT Electronics, Espoo', 'Espoo, Tech. Research Ctr.', 'Amateur Astron. Assoc., Seulaset', 'Picker Nordstar, Helsinki']


### Para la primera institución con 'number_of_papers > 0', obtenga el valor 'legacy_IC'.
Si ninguna institución satisface la condición number_of_papers > 0, escoja de nuevo otro país aleatorio y repita el proceso

In [519]:
## Obteniendo el numero de papers por institucion
names=np.array(inst_names)
num_papers = np.array([institution.get('metadata').get('number_of_papers') for institution in institution_link.json().get('hits').get('hits')])

legacy_ICN=names[num_papers>0][0] 

print("La primera institución con almenos 1 paper es:", legacy_ICN)

La primera institución con almenos 1 paper es: Helsinki, Academy of Finland


### B) Con el API de literatura obtenga el JSON con los artículos de menos de 10 autores usando el "legacy_ICN" de la siguiente manera

https://inspirehep.net/api/literature?sort=mostrecent&page=1&q=aff+Colombia,+U.+Natl.+and+ac+1->+10

aff: usa el valor de legacy_ICN
and: es un operador lógico
ac: establece los autores entre 1 y 10

In [520]:
#literature_link ="https://inspirehep.net/api/literature?sort=mostrecent&page=1&q=aff+{},+{}+and+ac+1->+10".format(pais,legacy_ICN.replace(' ','+'))
literature_link ="https://inspirehep.net/api/literature?sort=mostrecent&page=1&q=aff+{}+and+ac+1->+10".format(legacy_ICN.replace(' ','+'))
literature_link = requests.get(literature_link)    

### C) Para al menos un artículo de esa institución, extraiga el URL del perfil de cada autor de esa institución que se encuentra dentro del campo "authors" en "record" y luego en "$ref".
Por ejemplo https://inspirehep.net/api/authors/1010271

In [521]:
#Obteniendo los articulos 
articles = [article.get('metadata') for article in literature_link.json().get('hits').get('hits')]
#len(articles)

In [522]:
#Obteniendo los links de los perfiles de los autores
auth_profile = [author.get('record').get('$ref') for author in articles[0].get('authors')]
auth_profile

['https://inspirehep.net/api/authors/1074574',
 'https://inspirehep.net/api/authors/1027900',
 'https://inspirehep.net/api/authors/1279546',
 'https://inspirehep.net/api/authors/1616462']

### D) Con cada uno de los datos del resultado del API para cada perfil construya una tabla con los siguientes columnas (puede que alguno de los datos no esté disponible): 
Nombre Completo 
Correo electrónico
posición más reciente (la primera  que aparece en la lista "positions" del JSON) con su correspondiente:
rango 
institución 
fecha de inicio 
fecha de finalización

In [523]:
#Definiendo la funcion para extraer los datos del JSON
def auth_inf(auth_link):
    
    meta = requests.get(auth_link).json().get('metadata')

    name = meta.get('name').get('value')
    try: email = meta.get('email_addresses')[0].get('value') 
    except: email = 'None'    
    try: last_position = meta.get('positions')[0].get('rank') 
    except: last_position = 'None' 
    try: institution = meta.get('positions')[0].get('institution') 
    except: institution = 'None' 
    try: start = meta.get('positions')[0].get('start_date')
    except: start = 'None' 
    try: end = meta.get('positions')[0].get('end_date')
    except:  end  = 'None' 
    
    return name, email, last_position, institution, start, end

In [524]:
#Recorriendo los perfiles de cada autor
name = []
email = []
last_position = []
institution = []
start = []
end = []

for author in auth_profile:
    
    n,e,lp,i,s,en = auth_inf(author) 
    name.append(n)
    email.append(e)
    last_position.append(lp)
    institution.append(i)
    start.append(s)
    end.append(en)  

In [525]:
#Creando el diccionario para la tabla de pandas con los datos
dic = {'name':name,
       'email':email,
       'last_position':last_position,
       'institution':institution,
       'start':start,
       'end':end,
       }
dataframe = pd.DataFrame(dic) 
dataframe.head()

,name,email,last_position,institution,start,end
0,"Mäntysaari, Heikki",heikki.mantysaari@jyu.fi,JUNIOR,Jyvaskyla U.,2021,None
1,"Schenke, Björn Peter",bschenke@bnl.gov,SENIOR,Brookhaven,2017,None
2,"Shen, Chun",chunshen@wayne.edu,JUNIOR,Wayne State U.,2018,None
3,"Zhao, Wenbin",1501110084@pku.edu.cn,None,Hua-Zhong Normal U.,2020,2021
